# Capstone Project - Coursera
### This project noteboook is mainly for the capstone project in Coursera.


In [6]:
import pandas as pd
import numpy as np

In [7]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


---


### Kmeans


In [8]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

In [12]:
! pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

In [11]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

! pip install folium
import folium # map rendering library

print('Libraries imported.')

     |████████████████████████████████| 94 kB 7.0 MB/s  eta 0:00:01
Libraries imported.


In [101]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
dfs=pd.read_html(url)
df = dfs[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [102]:
df.rename(columns={"PostalCode": "Postal_Code", "Borough": "Borough", "Neighbourhood":"Neighbourhood"})
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Remove 'not assigned' in borough column

In [103]:
df.drop(df[df.Borough =='Not assigned'].index, inplace=True)
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [104]:
df.shape

(103, 3)

#### Check 'not assigned' in Neighbourhood column, using 'Parkwoods' as standard

In [105]:
df_Neighloc=df['Neighbourhood'].loc[df.Neighbourhood=='Parkwoods']
df_Neighloc[0:5]

2    Parkwoods
Name: Neighbourhood, dtype: object

In [106]:
df_Neighloc=df['Neighbourhood'].loc[df.Neighbourhood=='Not assigned']
df_Neighloc[0:5]

Series([], Name: Neighbourhood, dtype: object)

### Geocoder

In [107]:
geospatial = pd.read_csv("http://cocl.us/Geospatial_data")
#df_geo = pd.DataFrame(geospatial)
geospatial.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [108]:
df['Latitude'] = df['Postal Code'].map(geospatial.set_index('Postal Code')['Latitude'])
df['Longitude'] = df['Postal Code'].map(geospatial.set_index('Postal Code')['Longitude'])
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [119]:
df.groupby(['Borough']).mean()

,Latitude,Longitude
Borough,,
Central Toronto,43.701980,-79.398954
Downtown Toronto,43.654597,-79.383972
East Toronto,43.669436,-79.324654
East York,43.700303,-79.335851
Etobicoke,43.660043,-79.542074
Mississauga,43.636966,-79.615819
North York,43.750727,-79.429338
Scarborough,43.766229,-79.249085
West Toronto,43.652653,-79.449290


### Explore and cluster

In [131]:
#create df with only Boroughs containing the word Toronto
df_toronto = df[df['Borough'].isin(['Central Toronto', 'Downtown Toronto', 'East Toronto','West Toronto'])]
df_toronto

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
13,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
22,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
30,M4E,East Toronto,The Beaches,43.676357,-79.293031
31,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
40,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
41,M6G,Downtown Toronto,Christie,43.669542,-79.422564
49,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
50,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [133]:
colors = {'East Toronto' : 'red', 'Downtown Toronto' : 'red', 'Central Toronto':'red', 'West Toronto': 'red'}

map_osm = folium.Map(location=[43.654260, -79.360636], zoom_start=11)

df_toronto.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]], 
                                              radius=10, fill_color=colors[row['Borough']])
                                             .add_to(map_osm), axis=1)

map_osm